## 

- Same features as in 3a-trees
  - agr, bm, idiovol, mom12m, roeq
- Quantile transform features and ret in each cross-section
- Cross-validate on five years 2000-2004


In [ ]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import plotly.graph_objects as go

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor 

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select ticker, date, agr, bm, idiovol, mom12m, roeq, ret
    from data
    where date < '2005-01'
    """, 
    conn
)
features = ["agr", "bm", "idiovol", "mom12m", "roeq"]
df = df.dropna()

In [ ]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(output_distribution="normal")
def qtxs(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

df[features+["ret"]] = df.groupby(
  "date", 
  group_keys=False
)[features+["ret"]].apply(qtxs)

X = df[features]
y = df["ret"]

## GridSearchCV for Random Forest

. . .

```.p
model = RandomForestRegressor()

cv = GridSearchCV(
  model,
  param_grid={
    "max_depth": range(1, 11)
  },
)
_ = cv.fit(X, y)
```


In [ ]:
model = RandomForestRegressor()

cv = GridSearchCV(
  model,
  param_grid={
    "max_depth": range(1, 11)
  },
)
_ = cv.fit(X, y)

##


In [ ]:
results = pd.DataFrame(cv.cv_results_["params"])
results["score"] = cv.cv_results_["mean_test_score"]

trace = go.Scatter(
    x = results.max_depth,
    y = results.score,
    mode="lines+markers",
    marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
    hovertemplate="""
            max depth = %{x}<br>
            mean score = %{y:.1%}<extra></extra>
            """,
)
fig = go.Figure(trace)
fig.update_layout(
    title="Random Forest",
    xaxis_title="Max Depth",
    yaxis_title="Mean Test Score",
    yaxis_tickformat=".0%",
    template="plotly_white",
    xaxis_title_font_size=18,
    yaxis_title_font_size=18,
    font_size=16
)
fig.show()

## GridSearchCV for Multi-Layer Perceptron

. . .

```.p
param_grid={
    "hidden_layer_sizes": [
        [16, 8, 4, 2], 
        [16, 8, 4],
        [8, 4, 2], 
        [16, 8],
        [16, 4],
        [8, 4],
        [4, 4],
        [4, 2]
    ]
}
```

##

```.p
model = MLPRegressor(max_iter=500)
cv = GridSearchCV(
  model,
  param_grid=param_grid
)
_ = cv.fit(X, y)
```


In [ ]:
param_grid={
    "hidden_layer_sizes": [
        [16, 8, 4, 2], 
        [16, 8, 4],
        [8, 4, 2], 
        [16, 8],
        [16, 4],
        [8, 4],
        [4, 4],
        [4, 2]
    ]
}

model = MLPRegressor(max_iter=500)
cv = GridSearchCV(
  model,
  param_grid=param_grid
)
_ = cv.fit(X, y)

##


In [ ]:
results = pd.DataFrame(cv.cv_results_["params"])
results["score"] = cv.cv_results_["mean_test_score"]
results["number_layers"] = [len(x) for x in results.hidden_layer_sizes]
results["hidden_layer_sizes"] = [repr(x) for x in results.hidden_layer_sizes]

traces = []
for i in range(1, 5):
    trace = go.Scatter(
        x=results[results.number_layers==i].hidden_layer_sizes,
        y=results[results.number_layers==i].score,
        mode="markers",
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        name =f"{i} layer",
        hovertemplate="""
            hidden layer sizes = %{x}<br>
            mean score = %{y:.1%}<extra></extra>
        """
    )
    traces.append(trace)
fig = go.Figure()
for trace in traces:
    fig.add_trace(trace)
fig.update_layout(
    title="Multi-Layer Perceptron",
    xaxis_title="Hidden Layer Sizes",
    yaxis_title="Mean Test Score",
    yaxis_tickformat=".0%",
    template="plotly_white",
    xaxis_title_font_size=18,
    yaxis_title_font_size=18,
    font_size=16
)
fig.show()

## RandomizedSearchCV for Gradient Boosting

. . .

```.p
from scipy.stats import uniform
u = uniform(scale=0.2)

model = GradientBoostingRegressor()
cv = RandomizedSearchCV(
    model,
    param_distributions={
        "learning_rate": u,
        "max_depth": range(2, 10, 2)
    },
    n_iter=16
)
_ = cv.fit(X, y) 
```


In [ ]:
from scipy.stats import uniform
u = uniform(scale=0.2)

model = GradientBoostingRegressor()
cv = RandomizedSearchCV(
    model,
    param_distributions={
        "learning_rate": u,
        "max_depth": range(2, 10, 2)
    },
    n_iter=16
)
_ = cv.fit(X, y) 

##


In [ ]:
results = pd.DataFrame(cv.cv_results_["params"])
results["score"] = cv.cv_results_["mean_test_score"]
import numpy as np

depths = np.sort(results.max_depth.unique())
traces = []
for depth in depths:
    d = results[results.max_depth==depth]
    trace = go.Scatter(
        x=d.learning_rate,
        y=d.score,
        mode="markers",
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        hovertemplate="""
            max depth = %{depth}<br>
            learning rate = %{x:.2f}<br>
            mean score = %{y:.1%}<extra></extra>
            """,
        name = f"depth={depth}"
    )
    traces.append(trace)
fig = go.Figure()
for trace in traces: 
    fig.add_trace(trace)
fig.update_layout(
    title="Gradient Boosting Regressor",
    xaxis_title="Learning Rate",
    yaxis_title="Mean Test Score",
    yaxis_tickformat=".0%",
    template="plotly_white",
    xaxis_title_font_size=18,
    yaxis_title_font_size=18,
    font_size=16
)
fig.show()